In [175]:
import pandas as pd
import os
import xgboost

%run Dataloader.ipynb

 99%|██████████████████████████████████████▋| 6.00M/6.05M [00:06<00:00, 995kB/s]
100%|███████████████████████████████████████| 6.05M/6.05M [00:06<00:00, 984kB/s]


FileNotFoundError: File b'./train.csv' does not exist

In [145]:
val_pickles = ['PCA_Kmeans16_val.pkl',
        'PCA_Kmeans32_val.pkl',
        'VAE_val.pkl',
        'truncSVD_val.pkl']

train_pickles = ['PCA_Kmeans16_train.pkl',
          'PCA_Kmeans32_train.pkl',
          'VAE_train.pkl',
          'truncSVD_train.pkl']

In [146]:
def pickleLoader(list_of_pickles):
    #Init results dataframe
    df = pd.DataFrame()

    #Iterate through pickles and join
    for i in list_of_pickles:
        if len(df) == 0:
            df = pd.read_pickle(i)

        else:
            df = df.join(pd.read_pickle(i), how='left', lsuffix='', rsuffix=i[:5])

    #Change column names
    names = ['PCA_1', 'PCA_2', 'DNU_target1', 'Kmeans_16', 'DNU_PCA_1', 'DNU_PCA_2', 'DNU_target2', 'Kmeans_32',
             'VAE_1', 'VAE_2', 'DNU_targetsVAE', 'SVD_1', 'SVD_2', 'targets']
    df.columns = names
    #Drop DNU columns
    df = df.drop([i for i in df.columns if "DNU" in i], axis=1)
    
    return df

train_features = pickleLoader(train_pickles)
val_features = pickleLoader(val_pickles)

In [159]:
train_features.head()

,PCA_1,PCA_2,Kmeans_16,Kmeans_32,VAE_1,VAE_2,SVD_1,SVD_2
0,-4.289363,-0.668958,15,8,0.204750,-0.212566,6.742300,-5.324440
1,-4.390244,-1.762168,7,9,0.002832,-0.643341,0.858401,-0.707385
2,5.419955,-2.748868,7,19,0.570123,-0.700916,11.627939,2.617189
3,-2.843838,-1.037179,7,9,0.283977,-0.305720,3.058395,-0.936638
4,1.070864,-3.715246,7,23,2.844552,3.529658,5.706976,2.902905


In [160]:
val_features.head()

,PCA_1,PCA_2,Kmeans_16,Kmeans_32,VAE_1,VAE_2,SVD_1,SVD_2
0,-4.464939,1.691026,7,9,0.453804,-0.105982,1.229614,-0.632893
1,-2.134988,2.391655,7,9,0.575142,0.260778,3.862922,-0.098011
2,-3.790905,-1.868292,7,9,0.432332,1.108056,2.175072,-2.730801
3,-0.683397,3.374809,7,9,0.716456,0.974850,6.646363,-0.465280
4,-2.698667,-23.161974,13,2,1.296400,3.105117,11.987106,-21.725730


In [161]:
def add_features(old_features, new_features):
    old_features.index = new_features.index
#     new_features.drop('targets', axis=1, inplace = True)
    old_features = old_features.join(new_features)
    return old_features

extended_train_x = add_features(train_x, train_features)
extended_val_x = add_features(val_x, val_features)

In [174]:
from sklearn.metrics import log_loss

extended_train_x_ = xgboost.DMatrix(extended_train_x, label=train_y.values)